# 目标跟踪
![title](other_data/01.jpg)

# 机器人目标追踪

在这个示例中，我们将展示如何使用机器人进行对象跟踪!我们将使用一个在[COCO 数据集](http://cocodataset.org)上进行预训练的模型来检测90个不同的物体和1个背景类别。
包括：人 (索引 0),杯子 (索引 47)...等

目标检测区别于我们之前对整个图片的图像识别，而且单一的图像识别他的标签只有一个，比如之前学习的“避障”训练，他的标签就只有“有障碍”或者“无障碍”这样一个类别标签，然而目标检测除了类别标签之外，还有每个类别在图片里的位置和大小信息，而且每张图片还可能有多个类别和位置信息。当我们通过摄像头来检测时目标检测可以识别出图片里它所认识的所有物体并且会用一个矩形框把物体框在里面，在接下来的学习中你可以清楚的看到！你可以在[coco_index.txt](./coco_index.txt)文件中查看所有的类别和对应的索引。

## 1.导入我们需要的模块

In [ ]:
# from __future__ import division
from nxbot import Robot,event,bgr8_to_jpeg,pid
from modules.models import *
from modules.utils.util import *
from modules.utils.datasets import *
import threading
import os
import sys
import argparse
import cv2
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from traitlets.config.configurable import Configurable
from torch2trt import TRTModule

from modules.display_box import label_widget
from modules.deep_sort import build_tracker
from modules.utils.draw import draw_boxes
from modules.utils.parser import get_config
from modules.utils.log import get_logger
from modules.utils.io import write_results

## 2.参数设置

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--model_def", type=str, default="modules/config/yolov3-tiny.cfg", help="yolov3-tiny网络结构配置文件")
parser.add_argument("--Dachbot_model_path", type=str, default="../../../models/local/yolov3/object_following.weights", help="Dachbot模型文件")
parser.add_argument("--Dbot_model_path", type=str, default="../../../models/local/yolov3/object_following.engine", help="Dbot模型文件")
parser.add_argument("--deepsort_model_path", type=str, default="modules/config/deep_sort.yaml", help="deepsort配置文件")
parser.add_argument("--class_path", type=str, default="modules/data/coco.names", help="检测类别的所有种类")
parser.add_argument("--conf_thres", type=float, default=0.3, help="物体置信度")
parser.add_argument("--nms_thres", type=float, default=0.4, help="非极大抑制阈值")
parser.add_argument("--img_size", type=int, default=416, help="网络接收图片大小")
opt = parser.parse_args(args=[])
print(opt)

## 3.加载模型

In [ ]:
# 实例化机器人对象
rbt = Robot()
# 机器人名字
rbt_name = rbt.name

cfg = get_config()
# 加载跟踪模型配置文件
cfg.merge_from_file(opt.deepsort_model_path)

# 选择运行模型设备
if torch.cuda.is_available():
    device = torch.device("cuda")
    # 加载跟踪模型
    deepsort = build_tracker(cfg, use_cuda=True)
else:
    device = torch.device("cpu")
    # 加载跟踪模型
    deepsort = build_tracker(cfg, use_cuda=False)

# 如果是dachbot可以使用更准确的模型
if rbt_name=='dachbot':
    model = Darknet(opt.model_def, img_size=opt.img_size, TensorRT=False, Half=True).to(device).half()
    # 权重加载
    model.load_darknet_weights(opt.Dachbot_model_path)
    # Set in evaluation mode 前向推理时候会忽略 BatchNormalization 和 Dropout
    model.eval()
    
# 如果是dbot可以使用速度更快的模型    
elif rbt_name=='dbot':
    model_backbone = Darknet_Backbone(opt.model_def, img_size=opt.img_size).to(device).half()
    model = TRTModule()
    model.load_state_dict(torch.load(opt.Dbot_model_path))
    yolo_head = YOLOHead(config_path=opt.model_def)
    
# 提取可以识别的类别
classes = load_classes(opt.class_path)  # Extracts class labels from file
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

## 4.创建可视化窗口

In [ ]:
image_widget = widgets.Image(format='jpeg')
depth_image_widget = widgets.Image(format='jpeg')
speed_widget = widgets.FloatSlider(value=0.2, min=0.0, step=0.01, max=0.5, description='运行速度')
turn_gain_widget = widgets.FloatSlider(value=1, min=0.0, step=0.01, max=3.0, description='转向增益')
turn_dgain_widget = widgets.FloatSlider(value=0.0, min=0.0, step=0.01, max=2.0, description='回正微调')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='方向')
depth_slider = ipywidgets.FloatSlider(min=0.0, max=10000.0, description='深度值')

## 5.数据预处理

In [ ]:
def preprocess(image):
    image = np.array(Image.fromarray(image))
    imgTensor = transforms.ToTensor()(image)
    imgTensor, _ = pad_to_square(imgTensor, 0)
    imgTensor = resize(imgTensor, 416)
    imgTensor = imgTensor.unsqueeze(0)
    imgTensor = Variable(imgTensor.type(Tensor)).half()
    return imgTensor

## 6.检测模型是否能正常运行

我们通过numpy创建与我们将要预测的图片格式一致的形状为（416，416，3）的数组，这里我们创建的全为1的数组将这个数组经过预处理再将数据放入模型中，如果能运行通过说明模型可以正常使用了。

In [ ]:
try:
    img_data = np.ones([416, 416, 3],np.uint8)
    if rbt_name=='dachbot':
        model(preprocess(img_data))
    elif rbt_name=='dbot':
        model(preprocess(img_data))
except:
    print('请检查模型是否正确')

## 7.打开摄像头深度信息

In [ ]:
global depth
depth = 0
def on_new_depth(evt):
    time.sleep(0.05)
    depth_frame = np.asanyarray(evt.dict['data'].get_data())
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_frame, alpha=0.03), cv2.COLORMAP_JET)
    depth_colormap = cv2.resize(depth_colormap, (320,240))
    depth_image_widget.value = bgr8_to_jpeg(depth_colormap)
    
    global depth
    depth = evt.dict['data'].get_distance(310, 220)
    if depth ==0:
        depth = evt.dict['data'].get_distance(330, 225)


## 8.定义图像检测
在这里我们将通过目标检测模型进行检测，检测出图像中的所有物体，并将所有物体用矩形框将物体框出来，然后在所有物体中找出距离图像中心点最近的物体作为目标物体，如果没有找到目标就停止，如果找到目标机器人就像目标移动，然后根据目标与图像中心点的距离计算出机器人旋转的相对角度，让机器人跟着你走。

In [ ]:
# 随机选择颜色
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype="uint8")

global center_x
center_x = None
global detect_flag
detect_flag = True
import threading
global box_size
box_size = 0
def prediction():
    global detect_flag
    results = []
    frame_interval = 1
    idx_frame = 0
    last_steering = 0
    
    while detect_flag:
        global center_x
        global box_size
        detections = None
        origin_img = rbt.camera.read()
        if origin_img is not None:
            idx_frame += 1
            if idx_frame % frame_interval:
                continue
            image = cv2.cvtColor(origin_img,cv2.COLOR_BGR2RGB)
            imgTensor = preprocess(image)
            with torch.no_grad():
                
                detections = model(imgTensor)
                if rbt_name=='dbot':
                    detections = yolo_head(detections)
                # 非极大抑制筛选更加合适的候选框
                detections = non_max_suppression(detections, opt.conf_thres, opt.nms_thres)[0]
            
            if detections is not None:
                # 对预测类别框进行缩放
                bbox_xywh=[]
                cls_conf_list = []
                detections = rescale_boxes(detections, opt.img_size, origin_img.shape[:2])

                 # 目标跟踪类别名称
                choose_label = label_widget.children[0].children[1].value.strip()
                # 在图像上框出对应类别
                # 筛选出我们想要检测的对象
                for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
                    if choose_label == classes[int(cls_pred)]:
                        box_w = x2 - x1
                        box_h = y2 - y1
                        center_x = box_w/2+x1
                        center_y = box_h/2+y1
                        bbox_xywh.append([center_x, center_y, box_w, box_h])
                        cls_conf_list.append(cls_conf)
                
                bbox_xywh = np.asarray(bbox_xywh)
                cls_conf_list = np.asarray(cls_conf_list)
                if len(bbox_xywh)>0:
                    outputs = deepsort.update(bbox_xywh, cls_conf_list, image)
                    if len(outputs) > 0:
                        bbox_tlwh = []
                        bbox_xyxy = outputs[:, :4]
                        identities = outputs[:, -1]
                        ori_im = draw_boxes(origin_img, bbox_xyxy, identities)
                        for bb_xyxy in bbox_xyxy:
                            bbox_tlwh.append(deepsort._xyxy_to_tlwh(bb_xyxy))
                        results.append((idx_frame - 1, bbox_tlwh, identities))
                        center_x = (bbox_xyxy[0][2]-bbox_xyxy[0][0])/2+bbox_xyxy[0][0]
                        speed = speed_widget.value
                        steering = (0.5-center_x/300)*turn_gain_widget.value
                        steering = steering - (steering + last_steering)*turn_dgain_widget.value
                        steering_slider.value = steering
                        last_steering = steering
                        # 机器人腊肠狗控制
                        if rbt_name=='dachbot':
                            if depth < 1.0 and depth!=0:
                                rbt.base.move(-speed, 0, 0)
                            elif depth>1.2 and depth!=0:
                                rbt.base.move(speed, 0, steering)
                            else:
                                rbt.base.move(0,0,0)
                        # 机器人天秤座控制
                        elif rbt_name=='dbot':
                            if box_size > 0.5:
                                rbt.base.move(-speed, 0, 0)
                            elif box_size < 0.4:
                                rbt.base.move(speed, 0, steering)
                            else:
                                rbt.base.move(0,0,0)
                    else:
                        rbt.base.move(0,0,0)
                else:
                    rbt.base.move(0,0,0)    
            else:
                rbt.base.move(0,0,0)            
            origin_img = cv2.resize(origin_img, (320, 240), interpolation=cv2.INTER_CUBIC)
            image_widget.value=bgr8_to_jpeg(origin_img)
        else:
            rbt.base.move(0,0,0)
# 创建线程
process1 = threading.Thread(target=prediction,)

# 9.创建机器人运动线程

In [ ]:

# 创建机器人运动线程
def action():
    global detect_flag
    global center_x
    global box_size
    global depth
    depth_slider.value = depth
    last_steering = 0
    steering = None
    
    # PID参数
    xservo_pid = pid.PositionalPID(1.5, 0.002, 3)
    
    while detect_flag:
        time.sleep(0.05)
        speed = speed_widget.value
        if center_x is not None:
            steering0 = center_x/300*turn_gain_widget.value
            xservo_pid.SystemOutput = steering0
            xservo_pid.SetStepSignal(0.5)
            xservo_pid.SetInertiaTime(0.1, 0.06)
            steering = xservo_pid.SystemOutput
            steering_slider.value = steering
        # 机器人腊肠狗控制
        if rbt_name=='dachbot':
            if depth < 1.0 and depth!=0:
                rbt.base.move(-speed, 0, 0)
            elif depth>1.2 and depth!=0:
                if steering is not None:
                    rbt.base.move(speed, 0, steering)
                else:
                    rbt.base.move(0,0,0)
            else:
                rbt.base.move(0,0,0)
        # 机器人天秤座控制
        elif rbt_name=='dbot':
            if center_x is not None:
                if box_size > 0.5:
                    rbt.base.move(-speed, 0, 0)
                elif box_size < 0.4:
                    if steering is not None:
                        rbt.base.move(speed, 0, steering)
                    else:
                        rbt.base.move(0,0,0)
                else:
                    rbt.base.move(0,0,0)
            else:
                rbt.base.move(0,0,0)
        
        center_x = None
        steering = None
        
# 创建线程
process2 = threading.Thread(target=action,)

## 10.连接机器人进行实时检测

In [ ]:
rbt.connect()

if rbt_name=='dachbot':
    rbt.camera.start(enable_depth_stream=True)
    rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_DEPTH,on_new_depth)
    rbt.base.set_ptz(20)
    display(widgets.HBox([image_widget, depth_image_widget]))
    display(widgets.VBox([speed_widget,turn_gain_widget, label_widget]))
    display(steering_slider, depth_slider)
elif rbt_name=='dbot':
    rbt.camera.start()
    rbt.base.set_ptz(20)
    display(image_widget)
    display(widgets.VBox([speed_widget,turn_gain_widget, turn_dgain_widget, steering_slider]))
    display(label_widget)

process1.start()
process2.start()

## 11.断开与机器人的连接

In [ ]:
# detect_flag = False
# rbt.disconnect()